# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
import os
os.environ["MISTRAL_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy-mistral"  # If you don't set this, traces will go to the Default project

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [3]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("phd:19563583", include_model=True)

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [4]:
prompt

StructuredPrompt(input_variables=['question', 'subject'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'phd', 'lc_hub_commit_hash': '195635838118806eb01fb908c89911fcbce554c27568c324f775d0eb3290eb5e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='You have done your PhD in {subject}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'subject'], input_types={}, partial_variables={}, template='Answer this {question} under the context of your PhD in {subject}'), additional_kwargs={})], schema_={'title': 'extract_answer', 'description': 'Extract answer from the llm ersponse', 'type': 'object', 'properties': {'result': {'type': 'string', 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['result'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| Runn

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [5]:
hydrated_prompt = prompt.invoke({"question": "What is the speed of light?", "subject": "Physics"})
hydrated_prompt

{'result': "The speed of light in a vacuum, often denoted as **c**, is a fundamental constant of nature and plays a central role in physics, particularly in the theory of relativity. \n\n- **Numerical Value**: The speed of light is approximately **299,792,458 meters per second (m/s)**. This value is exact because the meter is defined based on the speed of light (since 1983, the International System of Units (SI) defines the meter as the distance light travels in a vacuum in 1/299,792,458 of a second).\n\n- **Significance in Physics**: \n  - In **classical electromagnetism**, the speed of light appears in Maxwell's equations, which describe how electric and magnetic fields propagate through space as electromagnetic waves.\n  - In **Einstein's theory of special relativity**, the speed of light is the ultimate speed limit for any information or matter in the universe. It is invariant, meaning it is the same for all observers regardless of their relative motion.\n  - In **general relativit

And now let's pass those messages to Mistral AI and see what we get back!

In [7]:
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

mistral_client = ChatMistralAI(model="mistral-large-latest")

# Debug: Let's see what the hydrated_prompt actually contains
print("Hydrated prompt type:", type(hydrated_prompt))
print("Hydrated prompt content:", hydrated_prompt)

# Convert the hydrated prompt to LangChain message format
messages = []

if isinstance(hydrated_prompt, dict):
    if "messages" in hydrated_prompt:
        # Format: {"messages": [...]}
        for msg in hydrated_prompt["messages"]:
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                messages.append(AIMessage(content=msg["content"]))
    elif "content" in hydrated_prompt:
        # Format: {"content": "text"}
        messages.append(HumanMessage(content=hydrated_prompt["content"]))
    else:
        # Try to extract any text content from the dict
        content = str(hydrated_prompt)
        messages.append(HumanMessage(content=content))
elif isinstance(hydrated_prompt, str):
    # Format: "text"
    messages.append(HumanMessage(content=hydrated_prompt))
else:
    # Fallback: convert to string
    messages.append(HumanMessage(content=str(hydrated_prompt)))

print("Final messages:", messages)
print("Number of messages:", len(messages))

if messages:
    response = mistral_client.invoke(messages)
    print("Response:", response)
else:
    print("No valid messages found to send to Mistral")

Hydrated prompt type: <class 'dict'>
Hydrated prompt content: {'result': "The speed of light in a vacuum, often denoted as **c**, is a fundamental constant of nature and plays a central role in physics, particularly in the theory of relativity. \n\n- **Numerical Value**: The speed of light is approximately **299,792,458 meters per second (m/s)**. This value is exact because the meter is defined based on the speed of light (since 1983, the International System of Units (SI) defines the meter as the distance light travels in a vacuum in 1/299,792,458 of a second).\n\n- **Significance in Physics**: \n  - In **classical electromagnetism**, the speed of light appears in Maxwell's equations, which describe how electric and magnetic fields propagate through space as electromagnetic waves.\n  - In **Einstein's theory of special relativity**, the speed of light is the ultimate speed limit for any information or matter in the universe. It is invariant, meaning it is the same for all observers re

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [9]:
# Pull a specific commit of the prompt - replace with actual commit hash
prompt = client.pull_prompt("phd:577bfacf")  # Replace with actual commit hash

Run this commit!

In [11]:
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

mistral_client = ChatMistralAI(model="mistral-large-latest")

hydrated_prompt = prompt.invoke({"question": "What is the speed of light?", "education": "high school", "subject": "Physics"})

# Convert the hydrated prompt to LangChain message format
messages = []

if isinstance(hydrated_prompt, dict):
    if "messages" in hydrated_prompt:
        # Format: {"messages": [...]}
        for msg in hydrated_prompt["messages"]:
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                messages.append(AIMessage(content=msg["content"]))
    elif "content" in hydrated_prompt:
        # Format: {"content": "text"}
        messages.append(HumanMessage(content=hydrated_prompt["content"]))
    else:
        # Try to extract any text content from the dict
        content = str(hydrated_prompt)
        messages.append(HumanMessage(content=content))
elif isinstance(hydrated_prompt, str):
    # Format: "text"
    messages.append(HumanMessage(content=hydrated_prompt))
else:
    # Fallback: convert to string
    messages.append(HumanMessage(content=str(hydrated_prompt)))

if messages:
    response = mistral_client.invoke(messages)
    print("Response:", response)
else:
    print("No valid messages found to send to Mistral")

Response: content='Given my context as someone whose highest level of education is high school with a focus on **Physics**, here’s how I would answer your question:\n\n---\n**The speed of light** is a fundamental constant in physics, denoted by the symbol **c**. Based on what I learned in high school physics:\n\n- The speed of light in a **vacuum** (like outer space) is approximately **299,792,458 meters per second** (or roughly **300,000 km/s**).\n- This value is exact because the meter is *defined* based on the speed of light (since 1983, the meter is the distance light travels in 1/299,792,458 of a second).\n- Light travels slower in other mediums (e.g., water, glass) due to interactions with atoms, but **c** always refers to its speed in a vacuum.\n\n**Why it matters in high school physics:**\n- It’s central to Einstein’s theory of **special relativity** (though we might only touch on basics like time dilation or \\(E=mc^2\\)).\n- It explains why nothing with mass can reach **c** (

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [14]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

phy_prompt = """You are a PhD level physicist who has specialized in string theory.
Any questions I ask you have to prove that you are a PhD, give detailed and technical answers.

Question: {question}
Topic: {topic}
Answer:"""

phy_template = ChatPromptTemplate.from_template(phy_prompt)
client.push_prompt("phy-rag-prompt", object=phy_template)

'https://smith.langchain.com/prompts/phy-rag-prompt/7077aa69?organizationId=bd531ccf-4286-4467-99ba-7eab707122af'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [15]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_mistralai import ChatMistralAI

client=Client()
model = ChatMistralAI(model="mistral-large-latest")

phy_prompt = """You are a PhD level physicist who has specialized in string theory.
Any questions I ask you have to prove that you are a PhD, give detailed and technical answers.

Question: {question}
Topic: {topic}
Answer:"""

phy_template = ChatPromptTemplate.from_template(phy_prompt)
chain = phy_template | model
client.push_prompt("phy-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/phy-runnable-sequence/a91ac07f?organizationId=bd531ccf-4286-4467-99ba-7eab707122af'